This program intends to optimize a portfolio utilising Markowitz Portfolio Theory
Start by...
1. Retrieve data from Yahoo! Finance
2. Clean data and calculate basic asset metrics
3. Create the covariance function w/constraints and minimize
4. Plot the min-variance frontier forcing different pofo returns
5. Plot the capital allocation line
6. Plot the indifference curves
7. Wrap it up with CFA utility maximization

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
tickers = "TYPE ASSET SYMBOLS HERE"
data = yf.download(tickers, period='3y', interval='1d')

'''We only need the adjusted close data; we can drop the rest'''
close = data.loc[:,'Adj Close'] 
rtns = close.pct_change()
n = len(close.columns)

In [ ]:
'''Calculate asset stats'''
mean_rtn = rtns.mean() * 252 ** 0.5
sd = np.std(rtns)
var = sd ** 2

In [ ]:
'''Define the porfolio covariance function and weight constraint'''
def covar(x):
    mtx = np.cov(rtns.dropna(), rowvar=False)
    for i in range(len(mtx)):
        for j in range(len(mtx)):
            mtx[i, j] = mtx[i, j] * x[i] * x[j]
    
    return mtx.sum()

def constraint(x):
    sum_weights = 1.0
    for i in range(len(x)):
        sum_weights = sum_weights - x[i]
    
    return sum_weights

In [ ]:
con = {'type' : 'eq', 'fun' : constraint}
x0 = [1 / n for i in range(n)]
'''Fill in list below for manual guesses'''
#x0 = []

'''Perform the optimization for min-variance portfolio'''
sol = minimize(covar, x0, method='SLSQP', constraints=con, options={'disp' : True})
print(sol.x, sum(sol.x))

In [ ]:
'''Calculate portfolio stats'''
p_var = covar(sol.x) * 252 ** 0.5
p_sd = p_var ** 0.5
p_rtn = (sol.x * mean_rtn).sum()

sharpe_ratio = (p_rtn / p_sd)

In [ ]:
'''Print Summary'''
print(f'Portfolio Statistics\nReturn: {round(p_rtn * 100, 3)}%\nVariance: {round(p_var, 5)}\nSt Dev: {round(p_sd * 100, 3)}%\nSharpe Ratio: {round(sharpe_ratio, 5)}')
x = 0
for i in close.columns:
    print(f'\nWeight in {i}: {round(sol.x[x] * 100, 3)}%')
    x += 1

Okay great, now we know the optimal weights for a min-variance portfolio. How can we expand on this?
Charting the minimum-variance frontier will give use a graphical representation of possible portfolio combinations. We can then synthesize this with the capital allocation line to solve for our final portfolio composition.